In [3]:
import numpy as np
import pandas as pd
data = pd.read_csv('./ml5/train.csv', sep=';', na_values='None')

FileNotFoundError: File b'./ml5/train.csv' does not exist

In [152]:
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [160]:
# Бинаризуем категориальные признаки 
CATEGORY_COLS = ['gender', 'cholesterol', 'gluc']
data = pd.get_dummies(data, columns=CATEGORY_COLS)

In [161]:
data['height_to_weight'] = data['height'] / data['weight']
data['ap_diff'] = data['ap_hi'] - data['ap_lo']
data['ap_ratio'] = data['ap_lo'] / data['ap_hi']
data.describe()

,id,age,height,weight,ap_hi,ap_lo,smoke,alco,active,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3,height_to_weight,ap_diff,ap_ratio
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,49999.500000,19464.864590,164.348140,74.180167,128.835960,81.506500,0.140530,0.110670,0.766252,0.649180,0.350820,0.747740,0.137300,0.114960,0.850150,0.074370,0.075480,2.287897,47.32946,0.645285
std,28867.657797,2470.428376,8.211187,14.379494,153.535263,10.082018,0.334384,0.303762,0.399155,0.477229,0.477229,0.434312,0.344166,0.318975,0.356926,0.262373,0.264166,0.409294,153.06387,0.059419
min,0.000000,8865.000000,50.000000,10.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.316940,2.00000,0.000000
25%,24999.750000,17662.000000,159.000000,65.000000,120.000000,80.000000,0.000000,0.000000,0.793578,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.012987,40.00000,0.615385
50%,49999.500000,19700.000000,165.000000,72.000000,120.000000,80.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.293333,40.00000,0.666667
75%,74999.250000,21324.000000,170.000000,82.000000,140.000000,90.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.538462,50.00000,0.666667
max,99999.000000,23713.000000,250.000000,200.000000,16020.000000,182.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.900000,15940.00000,0.980000


In [164]:
X_train = data.drop(['cardio', 'id'], 1).as_matrix()
y_train = data['cardio'].as_matrix()

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Разбиваем выборку на обучение и тест
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, 
                                                        y_train,
                                                        test_size=0.33, 
                                                        random_state=42)

LEARNING_RATE = 0.05
MAX_DEPTH = 5
SUBSAMPLE = 0.75
MAX_FEATURES = 0.5
LEAF = 8

# Создаем класс модели градиентного бустинга
clf = GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=LEARNING_RATE,
                                 max_depth=MAX_DEPTH, 
                                 subsample=SUBSAMPLE,
                                 max_features=MAX_FEATURES,
                                 min_samples_leaf=LEAF)

# Обучаем модель на обучающей выборке
clf.fit(X_train_, y_train_)

test_score = np.zeros((clf.n_estimators,), dtype=np.float64)
for i, y_pred in enumerate(clf.staged_predict_proba(X_test_)):
    test_score[i] = log_loss(y_test_, y_pred)
    
train_score = np.zeros((clf.n_estimators,), dtype=np.float64)
for i, y_pred in enumerate(clf.staged_predict_proba(X_train_)):
    train_score[i] = log_loss(y_train_, y_pred)

# Находим оптимальное число итераций на тесте
print(np.argmin(test_score), np.min(test_score))

# Выводим зависимость качества от числа итераций
plt.plot(range(len(train_score)), train_score, 'b-',
         range(len(test_score)), test_score, 'r-')

NameError: name 'X_train' is not defined